In [2]:
%reload_ext autoreload
%autoreload 2

In [7]:
from test_run_extracts import test_get_config, test_list_by_day, test_extract_events
import numpy as np
import pandas as pd
from pandas_summary import DataFrameSummary
from toolz import concatv
from pathlib import Path
from nbc_analysis.utils.debug_utils import runit

In [8]:
CONFIG_F = 'default'
config= runit(test_get_config)
config

>> created example config file '/Users/wmcabee/.config/nbc_analysis/config_example.yaml'
>> Using default config


{'VIDEO_END_BUCKET': 'nbc-event',
 'EVENT_BATCHES_D': '/Users/wmcabee/DATA/NBC2/batches',
 'PARTITIONS_D': '/Users/wmcabee/DATA/NBC2/partitions',
 'BATCH_LIMIT': 5,
 'LIMIT_EVENTS_PER_BATCH': 50000}

In [9]:
ret = runit(test_list_by_day)
ret

>> created example config file '/Users/wmcabee/.config/nbc_analysis/config_example.yaml'
>> Using default config
>> created example config file '/Users/wmcabee/.config/nbc_analysis/config_example.yaml'
>> Using default config
>> extract run, config={'LIMIT_EVENTS_PER_BATCH': 50000, 'VIDEO_END_BUCKET': 'nbc-event', 'EVENT_BATCHES_D': '/Users/wmcabee/DATA/NBC2/batches', 'PARTITIONS_D': '/Users/wmcabee/DATA/NBC2/partitions', 'BATCH_LIMIT': 5}, day_cnt=5
>> wrote /Users/wmcabee/DATA/NBC2/batches/ve_20190701_2019-09-30T00:21:27.075657Z.csv.gz,cnt=486
>> wrote /Users/wmcabee/DATA/NBC2/batches/ve_20190702_2019-09-30T00:21:27.794222Z.csv.gz,cnt=434
>> wrote /Users/wmcabee/DATA/NBC2/batches/ve_20190703_2019-09-30T00:21:28.214586Z.csv.gz,cnt=399
>> wrote /Users/wmcabee/DATA/NBC2/batches/ve_20190704_2019-09-30T00:21:28.501957Z.csv.gz,cnt=374
>> wrote /Users/wmcabee/DATA/NBC2/batches/ve_20190705_2019-09-30T00:21:28.787021Z.csv.gz,cnt=350
>> end extract run


In [303]:
%%time
ret = runit(test_extract_events)

>> created example config file '/Users/wmcabee/.config/nbc_analysis/config_example.yaml'
>> Using default config
>> start extract events, config={'VIDEO_END_BUCKET': 'nbc-event', 'EVENT_BATCHES_D': '/Users/wmcabee/DATA/NBC2/batches', 'PARTITIONS_D': '/Users/wmcabee/DATA/NBC2/partitions', 'BATCH_LIMIT': 5, 'LIMIT_EVENTS_PER_BATCH': 50000}
>> WARNING: limiting run to no more than 5 batches
>> start event download,batch_id=ve_20190701_2019-09-29T23:49:47.890503Z
>>downloading year=2019/month=07/event=video_end/NBCUniversal_20190701000737029958_74353.txt
>>downloading year=2019/month=07/event=video_end/NBCUniversal_20190701000932923891_18877.txt
>>downloading year=2019/month=07/event=video_end/NBCUniversal_20190701000934829386_48708.txt
>>downloading year=2019/month=07/event=video_end/NBCUniversal_20190701000940191147_31912.txt
>>downloading year=2019/month=07/event=video_end/NBCUniversal_20190701001416502005_12193.txt
>>downloading year=2019/month=07/event=video_end/NBCUniversal_201907010

In [10]:
%%time
indir = Path('/Users/wmcabee/DATA/NBC2/partitions/')
outfile = indir / 'merged.parquet.gz'

infiles = indir.glob('*')
df = pd.concat(map(pd.read_parquet, infiles))
#df.mvpd.value_counts(dropna=False,bins=20, sort=False)
df.sort_values(['mpid','event_start_unixtime_ms'], inplace=True)
df.to_parquet(outfile, index=False)

CPU times: user 3.82 s, sys: 651 ms, total: 4.47 s
Wall time: 3.81 s


In [11]:
len(df)

500000

In [12]:
dfs = DataFrameSummary(df)
dfs.columns_stats.T.sort_values('uniques',ascending=False)

,counts,uniques,missing,missing_perc,types
event_id,500000,249999,0,0%,categorical
event_start_unixtime_ms,500000,249743,0,0%,categorical
session_id,500000,150715,0,0%,categorical
session_start_unixtime_ms,500000,150661,0,0%,categorical
ip,500000,97316,0,0%,categorical
mpid,500000,79198,0,0%,numeric
resume_time,499780,12324,220,0.04%,categorical
video_id,500000,7011,0,0%,categorical
episode_title,500000,6846,0,0%,categorical
file,500000,1183,0,0%,categorical


In [13]:
dfs['video_type']

top             Full Episode: 474936
counts                        500000
uniques                           14
missing                            0
missing_perc                      0%
types                    categorical
Name: video_type, dtype: object

In [14]:
df.video_type.value_counts()

Full Episode         474936
Highlight             12838
Web Exclusive          4598
Sneak Peek             2892
Movie                  2498
Interview              1136
Current Preview         532
Excerpt                 256
None                    220
Behind the Scenes        68
Teaser Trailer            8
Commentary                8
Trailer                   6
Digital Episode           4
Name: video_type, dtype: int64